Auteur: F. Rosenblatt

Reference: F. Rosenblatt 1958 *The Perceptron: a Probabilistic Model for Information Storage and Organization in the Brain* Psychological Review, 65, 386-408

Le modéle est constitué des éléments suivants:
- des *unités sensitives (S-units)*: réagissent à un stimuli extérieur (lumière, son, touché, ...)
    - retournent `0` ou `1`:
        - `1` si le signal d'entrée dépasse un seuil $\theta$
        - `0` sinon
- des *unités d'associations (A-units)*
    - retournent `0` ou `1`:
        - `1` si la somme des signaux d'entrée dépasse un seuil $\theta$
        - `0` sinon
- des *unités de réponse (R-units)*: sortie du réseau
    - retournent `1`, `-1` ou une valeur indéterminée:
        - `1` si la somme des signaux d'entrée est positive
        - `-1` si elle est négative
        - une valeur indéterminée si elle est égale à 0
- une *matrice d'intéractions*


Calcul de $e_j$ :

$$
e_j = \sum s_i w_{ij} = si1 . wij1 + si2 . wij2 + si3 . wij3 + si4 . wij4
$$

Calcul du nouveau poid $w$ :

$$
w_{ij}(t+1) = w_{ij}(t) + n (d_j - s_j) s_i
$$

Nombre de neurones associatifs (A-units) :
    
    Nombre d'unités sensitives : 4
    
    Entrez le pas d'apprentissage ƞ :
ƞ : 1

    Entrez les poids de depart des synapses du réseau :
wij0 : 
    
    Entrez le motif à apprendre :
si1 : si2 : si3 : si4 : 

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import itertools

In [ ]:
def init_figure(size_x=10, size_y=5):
    fig, ax = plt.subplots(figsize=(size_x, size_y))
    ax.set_axis_off()
    ax.axis('equal')
    ax.set_xlim(-10, 20)   # TODO
    return fig, ax

def draw_neuron(axis,
                center,
                radius,
                fill_color='w',
                line_color='k',
                line_width=1,
                ag_func=None,
                tr_func=None):
    
    #circle = plt.Circle(center, radius, fill=True, color=fill_color, alpha=0)
    #axis.add_artist(circle)
    
    circle = patches.Circle(center,
                            radius=1,
                            fill=True,
                            edgecolor=line_color,
                            facecolor=fill_color)
    circle.set_zorder(20)  # put the circle on top
    axis.add_patch(circle)
    
    #circle = plt.Circle(center, radius, fill=False, color=line_color)
    #ax.add_artist(circle)

    x = center[0]
    y = center[1]
    
    line = mlines.Line2D([x, x],
                         [y - radius + 0.05, y + radius - 0.05],
                         lw=line_width,
                         color=line_color)
    line.set_zorder(21)
    axis.add_line(line)
    
    # Agregation function ######################
    
    if ag_func == "sum":
        line = mlines.Line2D([x - radius/4., x - 3 * radius/4., x - radius/2., x - 3. * radius/4., x - radius/4.],
                             [y + radius/4., y + radius/4., y, y - radius/4., y - radius/4.],
                             lw=line_width,
                             color=line_color)
        line.set_zorder(21)
        axis.add_line(line)
    
    # Transition function ######################
    
    if tr_func == "linear":
        line = mlines.Line2D([x + radius/4., x + 3. * radius/4.],
                             [y - radius/4., y + radius/4.],
                             lw=line_width,
                             color=line_color)
        line.set_zorder(21)
        axis.add_line(line)
    elif tr_func == "logistic":
        line = mlines.Line2D([x + radius/4., x + radius/2., x + radius/2., x + 3. * radius/4.],
                             [y - radius/4., y - radius/4., y + radius/4., y + radius/4.],
                             lw=line_width,
                             color=line_color)
        line.set_zorder(21)
        axis.add_line(line)

        
def draw_synapse(axis, p1, p2, color='k', line_width=1, label="", label_position=0.25, label_offset_y=0.3):
    line = mlines.Line2D([p1[0], p2[0]],
                         [p1[1], p2[1]],
                         lw=line_width,
                         color=color)
    line.set_zorder(10)
    axis.add_line(line)
    
    plt.text(x=p1[0] + label_position * (p2[0]-p1[0]),
             y=p1[1] + label_position * (p2[1]-p1[1]) + label_offset_y,
             s=label)


def draw_neural_network():
    fig, ax = init_figure()

    draw_synapse(ax, (-2, -6), (0, -6), label=r"$x_4$", label_position=0)
    draw_synapse(ax, (-2, -2), (0, -2), label=r"$x_3$", label_position=0)
    draw_synapse(ax, (-2,  2), (0,  2), label=r"$x_2$", label_position=0)
    draw_synapse(ax, (-2,  6), (0,  6), label=r"$x_1$", label_position=0)

    draw_synapse(ax, (0, -6), (10, 0), label=r"$w_4$", label_offset_y=-0.7)
    draw_synapse(ax, (0, -2), (10, 0), label=r"$w_3$", label_offset_y=-0.7)
    draw_synapse(ax, (0, 2),  (10, 0), label=r"$w_2$")
    draw_synapse(ax, (0, 6),  (10, 0), label=r"$w_1$")

    draw_synapse(ax, (10, 0), (12, 0), label_position=0.8, label=r"$y$")

    draw_neuron(ax, (0, -6), 1, tr_func="logistic")
    draw_neuron(ax, (0, -2), 1, tr_func="logistic")
    draw_neuron(ax, (0, 2),  1, tr_func="logistic")
    draw_neuron(ax, (0, 6),  1, tr_func="logistic")

    draw_neuron(ax, (10, 0), 1, ag_func="sum", tr_func="logistic")

draw_neural_network()

In [ ]:
w = np.array([0., 0., 0., 0., 2.])
nu = 1.

In [ ]:
def update_w(w, x, yd):
    s = np.sum(x * w)
    yo = 1. if s >= 0. else 0.
    w = w + nu * (yd - yo) * x
    print(x, s, yo, yd, w)
    return w

In [ ]:
examples = tuple(reversed(tuple(itertools.product((0., 1.), repeat=4))))

for it in range(4):
    print(it)
    for x in examples:
        yd = 1. if x == (1., 0., 0., 1.) else 0.
        w = update_w(w, np.array(x + (-1, )), yd)